# Chatbot Modeling

## Import Libs

In [14]:
# General
import json
import string
import pickle
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import LabelEncoder

# Deep Learning
import tensorflow as tf

# Word Preprocessing
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt') # Only needed for first time usage of NLTK

[nltk_data] Downloading package punkt to C:\Users\Judan Syamsul
[nltk_data]     Hadad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load Files for Preprocessing and Modeling

### Intent

In [15]:
# Load json file
f = open('intent/intent.json', 'r')
intent_json = json.load(f)

In [16]:
# Create list from json
input = []
intent = []

for i in range(len(intent_json['intents'])):
    for user_input in intent_json['intents'][i]['input']:
        input.append(user_input)
        intent.append(intent_json['intents'][i]['intent'])

In [17]:
# Create dataframe from json
df = pd.DataFrame({
    'input': input,
    'intent' : intent
    # 'response' : response
})
df.head()

,input,intent
0,halo,sapa
1,hei,sapa
2,hey,sapa
3,hello,sapa
4,hallo,sapa


### Stopwords

In [18]:
# Define stemmer and stopword
factory = StemmerFactory()
stemmer = factory.create_stemmer()

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

In [19]:
stopwords.remove('ok')
stopwords.remove('oh')
stopwords.remove('tidak')
stopwords.remove('ya')

### Slang

In [20]:
# slang = pd.read_csv('lexicon/slang ke semi baku.csv')

# slang_replace = {}
# for i, row in enumerate(slang['slang']):
#     slang_replace[row] = slang['formal'].iloc[i]

In [21]:
# baku = pd.read_csv('lexicon/slang ke baku.csv')

# std_word_replace = {}
# for i, row in enumerate(baku['slang']):
#     std_word_replace[row] = baku['baku'].iloc[i]

In [22]:
baku = pd.read_csv('lexicon/baku.csv')

In [23]:
std_word_replace = {}
for i, row in enumerate(baku['slang']):
    std_word_replace[row] = baku['baku'].iloc[i]

In [24]:
std_word_replace['gak']

'tidak'

## Preprocessing

In [25]:
# Create text cleaning function
def clean_text(text):
    new_text = []
    text = text.lower() # Lowercase
    # Loop each word in a sentence
    for kata in text.split(): 
        # Keep word not in slang or standard word
        if kata not in std_word_replace: 
            new_text.append(kata) 
        # Replace non-formal word with standard word
        elif kata in std_word_replace:
            new_text+=std_word_replace[kata].split() 
    # Join words without stopwords after stemming
    new_text = ' '.join(
        stemmer.stem(word) for word in new_text if word not in stopwords
    )
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    return new_text

In [26]:
kalimat = 'Aku laper banget gatau maunya makan jeruk apa lagi meniru-niru... daah ngelamar'
clean_text(kalimat)

'aku lapar banget tidak tahu mau makan jeruk apa tiru deh lamar'

In [27]:
df['clean_input'] = df['input'].apply(clean_text)
df['clean_input'].head()

0     halo
1      hei
2      hai
3    hello
4     halo
Name: clean_input, dtype: object

## Create Corpus

In [28]:
# Create corpus
words = set([
    word for word in df['clean_input'] for word in word_tokenize(word)
    ]
)

In [29]:
corpus_size = len(words)

In [30]:
# Calculate word length of each row
df['length'] = df['clean_input'].apply(word_tokenize).apply(len)

In [31]:
sequence_length = int(round(df['length'].max(),0))

In [32]:
corpus_size, sequence_length

(328, 8)

## Label Encoding

In [33]:
# Label encoding
le = LabelEncoder()
y_train = le.fit_transform(df['intent'])
y_train = tf.keras.utils.to_categorical(y_train)

In [34]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [35]:
len(le.classes_)

22

## Modeling

### Text Vectorization

In [36]:
textvect = tf.keras.layers.TextVectorization(
    max_tokens=corpus_size,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=sequence_length
)
textvect.adapt(df['clean_input'])

In [37]:
tes = 'saya mau daftar rekrutmen'
textvect(clean_text(tes))

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([19, 12,  9,  0,  0,  0,  0,  0], dtype=int64)>

### Word Embedding

In [38]:
embedding = tf.keras.layers.Embedding(
    input_dim=corpus_size,
    output_dim=16,
    input_length=sequence_length,
    embeddings_initializer='uniform'
)

In [39]:
embedding(textvect(clean_text(tes)))

<tf.Tensor: shape=(8, 16), dtype=float32, numpy=
array([[ 0.02719415, -0.00168892,  0.02013149, -0.02542167,  0.03772203,
        -0.00469325,  0.00458628,  0.04243712,  0.04297456,  0.03012897,
         0.0426373 ,  0.00617666,  0.02236391,  0.0212539 , -0.03604666,
        -0.00142021],
       [-0.0126008 ,  0.03160738,  0.02876427, -0.03624273,  0.01037991,
        -0.00385446, -0.04692028,  0.04453794, -0.04648004,  0.00592103,
         0.04626051,  0.02893097,  0.04876984,  0.00549614, -0.00620693,
        -0.03576725],
       [ 0.02888567,  0.00938115, -0.04323957,  0.02845827,  0.01413968,
         0.00577967, -0.00314509, -0.04831542,  0.03514165,  0.01202588,
        -0.01897939,  0.04379736, -0.01736676,  0.04582937,  0.01396929,
         0.04885042],
       [-0.0366608 , -0.01916505,  0.01627165, -0.00037545,  0.00708333,
         0.0417504 , -0.02539539,  0.00917493,  0.02663914,  0.02248241,
         0.02416353,  0.04191277,  0.04211726, -0.04844122, -0.01665397,
         

### Layering

In [40]:
# Create model
input = tf.keras.layers.Input(shape=(1,), dtype='string')
hidden_layer = textvect(input)
hidden_layer = embedding(hidden_layer)
hidden_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16))(hidden_layer)
output = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(hidden_layer)
model = tf.keras.Model(inputs=input, outputs=output)

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

### Training

In [41]:
hist = model.fit(df['clean_input'], y_train, epochs=100, verbose=0)
pd.DataFrame(hist.history).tail()

,loss,categorical_accuracy
95,0.023645,1.0
96,0.024016,1.0
97,0.022329,1.0
98,0.021639,1.0
99,0.021464,1.0


### Evaluating Model

In [42]:
model.evaluate(df['clean_input'], y_train)

16/16 [==============================] - 1s 1ms/step - loss: 0.0207 - categorical_accuracy: 1.0000


[0.02069256640970707, 1.0]

In [43]:
y_train_df = pd.DataFrame(y_train, columns=le.classes_)
y_train_df['intent'] = y_train_df.idxmax(axis=1)

model_pred = model.predict(df['clean_input'])
model_pred = pd.DataFrame(model_pred, columns=le.classes_)
model_pred['intent'] = model_pred.idxmax(axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_train_df['intent'], model_pred['intent']))

                  precision    recall  f1-score   support

              CV       1.00      1.00      1.00        12
           biaya       1.00      1.00      1.00        27
          daftar       1.00      1.00      1.00        22
         dokumen       1.00      1.00      1.00        23
    error upload       1.00      1.00      1.00         9
             ktp       1.00      1.00      1.00        17
     link sosmed       1.00      1.00      1.00        17
        lowongan       1.00      1.00      1.00        17
   lupa password       1.00      1.00      1.00        21
            nama       1.00      1.00      1.00         9
        pas foto       1.00      1.00      1.00        16
       pengantar       1.00      1.00      1.00        10
         penutup       1.00      1.00      1.00        48
   qualification       1.00      1.00      1.00        21
responsibilities       1.00      1.00      1.00        33
          salary       1.00      1.00      1.00        37
            s

### Function Covering End-to-End Process

In [44]:
def bot_response(text):
    """Take text as function input then predict using model. Return response based on highest probability using numpy argmax    
    """
    text = clean_text(text)
    pred = model.predict([text])
    res = le.classes_[pred.argmax()] # Get the index with highest probability
    i = 0
    try:
        if textvect(text).numpy().max() > 1: # If the input is known word(s)
            while i < len(intent_json['intents']):
                if res == intent_json['intents'][i]['intent']:
                    response = intent_json['intents'][i]['response']
                    break
                else:
                    i+=1
        else: # If only unknown word(s)
            response = ['Maaf, Kak. Aku tidak mengerti...']
    except: # If empty string or any error occured
        response = ['Maaf, Kak. Aku tidak mengerti...']

    # For debugging only
    dict_temp = []
    for i in range(len(pred[0])):
        temp = {le.classes_[i]: pred[0][i]}
        dict_temp.append(temp)
    print(dict_temp)
    print(le.classes_[pred.argmax()])

    return print(np.random.choice(response))

In [45]:
tes1 = 'aku mau daftar rekrutmen'
clean_text(tes1)

'aku mau daftar rekrutmen'

In [46]:
model.predict([clean_text(tes1)])

array([[4.25674989e-05, 2.58333807e-06, 9.88528907e-01, 3.82413040e-03,
        2.51323399e-05, 1.67314764e-04, 6.20685343e-04, 4.64891811e-04,
        1.77134098e-05, 2.97432160e-03, 1.79170036e-06, 5.21104412e-05,
        2.22422808e-04, 1.41829834e-03, 1.02728244e-07, 3.79859011e-06,
        2.12452469e-05, 1.29062914e-06, 3.55690503e-07, 1.51536544e-03,
        7.23314151e-05, 2.26853281e-05]], dtype=float32)

In [47]:
model.predict([clean_text(tes1)]).argmax()

2

In [48]:
le.classes_[model.predict([clean_text(tes1)]).argmax()]

'daftar'

In [49]:
le.classes_[model.predict([clean_text(tes1)]).argmax()] == intent_json['intents'][2]['intent']

True

In [50]:
intent_json['intents'][2]['response']

['Untuk mengikuti rekrutmen ini, mohon ikuti langkah berikut ya, Kak:\n1. Baca tata cara melamar di https://rekrutmen.fiktif.id/tatacara\n2. Buat akun di https://rekrutmen.fiktif.id/registrasi menggunakan alamat email dan nomor telepon aktif milik pribadi\n3. Lengkapi formulir pendaftaran dan upload seluruh dokumen yang dibutuhkan dan tunggu pengumuman tahap berikutnya di https://rekrutmen.fiktif.id/news atau email masing-masing']

In [51]:
bot_response('aku mau daftar rekrutmen')

[{'CV': 4.25675e-05}, {'biaya': 2.583338e-06}, {'daftar': 0.9885289}, {'dokumen': 0.0038241304}, {'error upload': 2.513234e-05}, {'ktp': 0.00016731476}, {'link sosmed': 0.00062068534}, {'lowongan': 0.0004648918}, {'lupa password': 1.771341e-05}, {'nama': 0.0029743216}, {'pas foto': 1.7917004e-06}, {'pengantar': 5.211044e-05}, {'penutup': 0.00022242281}, {'qualification': 0.0014182983}, {'responsibilities': 1.02728244e-07}, {'salary': 3.7985901e-06}, {'sapa': 2.1245247e-05}, {'sertifikat': 1.2906291e-06}, {'skck': 3.556905e-07}, {'timeline': 0.0015153654}, {'training': 7.2331415e-05}, {'transkrip-ijazah': 2.2685328e-05}]
daftar
Untuk mengikuti rekrutmen ini, mohon ikuti langkah berikut ya, Kak:
1. Baca tata cara melamar di https://rekrutmen.fiktif.id/tatacara
2. Buat akun di https://rekrutmen.fiktif.id/registrasi menggunakan alamat email dan nomor telepon aktif milik pribadi
3. Lengkapi formulir pendaftaran dan upload seluruh dokumen yang dibutuhkan dan tunggu pengumuman tahap berikutny

### Saving Model

In [52]:
import pickle
pickle.dump(le, open('saved_model/encoder.pkl', 'wb'))

In [53]:
pickle.dump({'config': textvect.get_config(),
             'weights': textvect.get_weights()}
            , open("saved_model/textvect.pkl", "wb"))

In [54]:
model.save('saved_model/model', save_traces=True)

INFO:tensorflow:Assets written to: saved_model/model\assets


INFO:tensorflow:Assets written to: saved_model/model\assets


## Inference

In [55]:
textvect('ok')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([245,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [56]:
clean_text('ok')

'ok'

In [57]:
textvect('ok').numpy().max() > 1

True

In [58]:
bot_response('a')

[{'CV': 0.0051773656}, {'biaya': 0.0031323435}, {'daftar': 0.00022532848}, {'dokumen': 0.00049814914}, {'error upload': 3.0471535e-06}, {'ktp': 8.621317e-05}, {'link sosmed': 0.0003759128}, {'lowongan': 0.000106551895}, {'lupa password': 0.0003127815}, {'nama': 0.00014735002}, {'pas foto': 3.934829e-05}, {'pengantar': 4.3081e-07}, {'penutup': 0.9589822}, {'qualification': 0.0003491384}, {'responsibilities': 2.330609e-05}, {'salary': 0.00023143816}, {'sapa': 0.023377875}, {'sertifikat': 2.1739683e-05}, {'skck': 0.00027322245}, {'timeline': 0.003578926}, {'training': 0.0005294068}, {'transkrip-ijazah': 0.0025280046}]
penutup
Maaf, Kak. Aku tidak mengerti...


In [59]:
bot_response('ya')

[{'CV': 0.0017228933}, {'biaya': 0.0012969999}, {'daftar': 0.00010557187}, {'dokumen': 0.00023918864}, {'error upload': 9.860624e-07}, {'ktp': 1.817531e-05}, {'link sosmed': 0.00016077337}, {'lowongan': 5.481459e-05}, {'lupa password': 0.00012702463}, {'nama': 3.0539155e-05}, {'pas foto': 5.517382e-06}, {'pengantar': 6.2392715e-08}, {'penutup': 0.9875542}, {'qualification': 8.971097e-05}, {'responsibilities': 2.7754597e-06}, {'salary': 7.842839e-05}, {'sapa': 0.003901662}, {'sertifikat': 1.920295e-06}, {'skck': 4.5028963e-05}, {'timeline': 0.0029251098}, {'training': 0.00036178785}, {'transkrip-ijazah': 0.0012769953}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [60]:
testext = 'jeruk'
clean_text(testext), textvect(clean_text(testext))

('jeruk',
 <tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>)

In [61]:
textvect(clean_text('laper pengen makan jeruk')).numpy().max() > 1

False

In [62]:
bot_response('oh ok')

[{'CV': 0.002917032}, {'biaya': 0.0012345137}, {'daftar': 7.668769e-05}, {'dokumen': 0.0003272902}, {'error upload': 1.8652772e-06}, {'ktp': 0.000102716986}, {'link sosmed': 0.00032886682}, {'lowongan': 5.0391965e-05}, {'lupa password': 0.00014276136}, {'nama': 8.3849874e-05}, {'pas foto': 7.2652742e-06}, {'pengantar': 5.4166463e-07}, {'penutup': 0.98745614}, {'qualification': 0.000784242}, {'responsibilities': 9.010748e-06}, {'salary': 0.00039186567}, {'sapa': 0.0034138167}, {'sertifikat': 2.2284086e-05}, {'skck': 0.00023707659}, {'timeline': 0.0008456431}, {'training': 0.00010258267}, {'transkrip-ijazah': 0.001463449}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [63]:
bot_response('ada lowongan apa nih di pt fiktif')

[{'CV': 0.002873956}, {'biaya': 8.890966e-05}, {'daftar': 0.022975022}, {'dokumen': 0.0029747968}, {'error upload': 0.008007164}, {'ktp': 0.01696796}, {'link sosmed': 0.0019346797}, {'lowongan': 0.85693395}, {'lupa password': 0.001204995}, {'nama': 0.015900232}, {'pas foto': 4.3455133e-05}, {'pengantar': 0.02386694}, {'penutup': 0.0013333951}, {'qualification': 0.040605422}, {'responsibilities': 7.988949e-05}, {'salary': 2.2605844e-05}, {'sapa': 0.0008909284}, {'sertifikat': 0.0002591312}, {'skck': 8.872629e-05}, {'timeline': 0.0016363872}, {'training': 2.6321279e-05}, {'transkrip-ijazah': 0.0012850772}]
lowongan
Saat ini PT Fiktif sedang membuka lowongan untuk posisi Data Analyst, Data Scientist, dan Data Engineer. Yuk daftar lewat https://rekrutmen.fiktif.id/


In [64]:
bot_response('ingin')

[{'CV': 0.0015265091}, {'biaya': 0.0013876593}, {'daftar': 0.00016374185}, {'dokumen': 0.000870736}, {'error upload': 5.809087e-07}, {'ktp': 3.6238012e-05}, {'link sosmed': 0.00027143568}, {'lowongan': 9.9529214e-05}, {'lupa password': 0.0003889023}, {'nama': 3.152806e-05}, {'pas foto': 3.740186e-06}, {'pengantar': 1.0274791e-07}, {'penutup': 0.98144126}, {'qualification': 0.00016042846}, {'responsibilities': 1.252388e-05}, {'salary': 4.514851e-05}, {'sapa': 0.008506802}, {'sertifikat': 2.8603856e-06}, {'skck': 8.133128e-05}, {'timeline': 0.003986026}, {'training': 0.00052375434}, {'transkrip-ijazah': 0.00045915137}]
penutup
Maaf, Kak. Aku tidak mengerti...


In [65]:
bot_response('pas foto hilang gimana ya')

[{'CV': 0.000159947}, {'biaya': 0.00018639004}, {'daftar': 4.360997e-05}, {'dokumen': 1.7687007e-07}, {'error upload': 0.00024153096}, {'ktp': 4.602501e-06}, {'link sosmed': 3.0998952e-05}, {'lowongan': 4.5161866e-07}, {'lupa password': 1.2468522e-06}, {'nama': 9.044067e-05}, {'pas foto': 0.99385935}, {'pengantar': 4.369212e-06}, {'penutup': 2.2266317e-05}, {'qualification': 4.985285e-07}, {'responsibilities': 0.00026670712}, {'salary': 0.0020211367}, {'sapa': 0.00025502458}, {'sertifikat': 0.00012338124}, {'skck': 3.6893594e-05}, {'timeline': 0.00011732747}, {'training': 0.0011761286}, {'transkrip-ijazah': 0.0013574868}]
pas foto
Pas foto merupakan dokumen wajib pendaftaran ya, Kak.

Berikut ketentuan pas foto yang harus dipenuhi:
- Background berwarna biru dengan pakaian formal (kemeja dan/atau jas)
- Foto berukuran 3x4 dan berwarna
- Disarankan untuk menggunakan foto terbaru
- Upload foto di https://rekrutmen.fiktif.id/dokumen dengan ukuran file tidak lebih dari 1 MB dengan format f

ktp, pas foto, password masih ketuker2

In [66]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [67]:
textvect('loker')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([161,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [68]:
textvect('wis')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>

In [69]:
data = {'user_input':'hai hai'}
data = json.dumps(data)
loaded_data = json.loads(data)

In [70]:
type(loaded_data)

dict

In [71]:
tespred = str(model.predict([clean_text(loaded_data['user_input'])]).argmax())

In [72]:
tes_res = {'pred':tespred}
tes_res = json.dumps(tes_res)
loaded_res = json.loads(tes_res)

In [73]:
le.classes_[int(loaded_res['pred'])]

'sapa'

In [74]:
loaded_data['user_input']

'hai hai'

In [75]:
1000

1000

In [76]:
str(1000)

'1000'

In [77]:
int(str(1000))

1000

In [78]:
le.classes_[int(str(10))]

'pas foto'

In [79]:
textvect('loker')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([161,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [80]:
user_input = 'apa kualifikasi engineer'
clean_input = clean_text(user_input)
pred = model.predict([clean_input]).argmax()
if textvect(clean_input).numpy().max() <= 1:
    label_idx = 1000
else:
    label_idx = pred 
if label_idx != 1000:
    i = 0
    while i < len(intent_json['intents']):
        if le.classes_[label_idx] == intent_json['intents'][i]['intent'] and (le.classes_[label_idx] != 'responsibilities' or le.classes_[label_idx] != 'qualification'):
            response = intent_json['intents'][i]['response']
            break
        elif le.classes_[label_idx] == 'responsibilities':
            if any(x in clean_input for x in ['scientist', 'ds']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Scientist:\n- Merancang dan mengembangkan berbagai solusi Machine Learning dan Deep Learning untuk meningkatkan pengalaman pengguna bagi konsumen\n- Berkolaborasi dengan seluruh elemen bisnis dan bertanggung jawab untuk merencanakan solusi end-to-end berbasis data untuk menyelesaikan permasalahan bisnis\n- Menjadi thinking partner bagi stakeholder lain untuk memperbaiki alur perjalanan data dan penggunaannya dalam operasional perusahaan, misal merancang proses feedback-loop atau human-in-the-loop untuk meningkatkan performa model secara berkelanjutan"]
                break
            elif any(x in clean_input for x in ['engineer', 'de']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Engineer:\n- Membangun dan menjaga end-to-end data pipeline dari input dan output heterogen\n- Menangani dan mengelola data warehouse\n- Membantu tim mentransformasikan data (ETL) dan mengembangkan proses ETL dari beberapa sumber\n- Menganalisis dan mengorganisir data mentah \n- Memastikan kualitas data dan integrasi data"]
                break
            elif any(x in clean_input for x in ['analyst', 'da', 'analis']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Analyst:\n- Mengumpulkan dan menyediakan data untuk membantu stakeholder lain meningkatkan metrik bisnis perusahaan dan retensi pelanggan\n- Menganalisis data untuk menemukan insight yang dapat ditindaklanjuti seperti membuat funnel conversion analysis, cohort analysis, long-term trends, user segmentation, dan dapat membantu meningkatkan kinerja perusahaan dan mendukung pengambilan keputusan yang lebih baik\n - Mengidentifikasi kebutuhan dan peluang bisnis berdasarkan data yang tersedia"]
                break
        elif le.classes_[label_idx] == 'qualification':
            if any(x in clean_input for x in ['scientist', 'ds']):
                response = ["Untuk posisi Data Scientist ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik, Machine Learning, Deep Learning untuk menyelesaikan permasalahan bisnis\n3. Memiliki pengalaman kerja di bidang Data Science selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif any(x in clean_input for x in ['engineer', 'de']):
                response = ["Untuk posisi Data Engineer ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pengalaman bekerja dengan tools untuk ETL seperti AWS Glue, SSIS, Informatica, dll.\n3. Memiliki pengalaman kerja di bidang Data Engineer selama 1-3 tahun\n4. Memiliki pemahaman yang baik tentang ETL, SQL, dan noSQL\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif any(x in clean_input for x in ['analyst', 'da', 'analis']):
                response = ["Untuk posisi Data Analyst ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik dan Inferensial untuk mencari peluang bisnis\n3. Memiliki pengalaman kerja di bidang Data Analyst selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data serta visualisasi dengan tools seperti Power BI, Tableau, dll.\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
        else:
            i+=1
else:
    response = ['Maaf, Kak. Aku tidak mengerti chatnya...']

print(np.random.choice(response))

Untuk posisi Data Engineer ada beberapa kualifikasi yang harus dipenuhi:
1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan
2. Memiliki pengalaman bekerja dengan tools untuk ETL seperti AWS Glue, SSIS, Informatica, dll.
3. Memiliki pengalaman kerja di bidang Data Engineer selama 1-3 tahun
4. Memiliki pemahaman yang baik tentang ETL, SQL, dan noSQL
5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik


In [81]:
le.classes_[label_idx]

'qualification'